<font size=18>Lesson 04: Quadratic Programming and Local Optimization</font>

In [3]:
import sys
sys.path

['/Users/ebolotin/Dropbox/techdev/uwl/775/lesson_4/Lesson 04 Presentation - QP NP',
 '/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python37.zip',
 '/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload',
 '',
 '/Users/ebolotin/Library/Python/3.7/lib/python/site-packages',
 '/usr/local/lib/python3.7/site-packages',
 '/usr/local/Cellar/numpy/1.16.4_1/libexec/nose/lib/python3.7/site-packages',
 '/usr/local/lib/python3.7/site-packages/IPython/extensions',
 '/Users/ebolotin/.ipython']

In [ ]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));

In [5]:
# imports
%matplotlib notebook
import numpy as np
from scipy import interpolate
from scipy.optimize import minimize
import babel.numbers as numbers
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import json
import time
import seaborn as sns
sns.set_style("darkgrid")
# from mpl_toolkits.basemap import Basemap
import mpl_toolkits
import ipopt

ModuleNotFoundError: No module named 'ipopt'

# Optimization Basics (video)

Work your way through the embedded storybook below to learn some basic ideas about optimization.  This material complements the material in the textbook.  The graphs and demos in Slides 9-11 are available in the separate file Storybook_Graphs_04.ipynb.

In [ ]:
# execute this cell for video
from IPython.display import IFrame
IFrame(
    "https://media.uwex.edu/content/ds/ds775_r19/ds775_lesson4-optimization-basics/index.html",
    width=900,
    height=600)

# Quadratic Programming

You should read about quadratic programming in the textbook.  In short, the constraints are the same as they are in linear programming and the objective function can have degree 2 and interaction terms.  In Pyomo it is only a matter of changing the solver to one capable of solving quadratic programs, `ipopt`, instead of `glpk`.  Other solvers like CPLEX, a commercial solver, could also be used.  You  may need to install `ipopt` using conda on your own machine.

## Wyndor Example

We'll use Pyomo to solve the quadratic variation of the Wyndor problem illustrated in Figure 13.6 on page 554.

<img src="images/wyndor_quad.png" width="600">

This profit function is kind of nonsensical, but it will serve to illustrate a Pyomo solution.  We'll use a concrete model formulation for simplicity.  Note, the Pyomo package `minimize` conflicts with the `minimize` from `scipy.optimize` we're using elsewhere in this notebook, so we'll import `pyomo` a bit differently here than usual.  Alternately we could re-import `minimize` from `scipy.optimize` after we're done with Pyomo.

In [ ]:
# unfold to see Pyomo solution for Wyndor Quadratic Program
import pyomo.environ as pyo

# Concrete Model
model = pyo.ConcreteModel(name="Wyndor")

products = ['drs', 'wdw']

bounds_dict = {'drs': (0, 4), 'wdw': (0, 6)}


def bounds_rule(model, product):
    return (bounds_dict[product])


model.x = pyo.Var(products, domain=pyo.Reals, bounds=bounds_rule)

# Objective
model.profit = pyo.Objective(expr=126.0 * model.x['drs'] -
                         9.0 * model.x['drs']**2 + 182.0 * model.x['wdw'] -
                         13.0 * model.x['wdw']**2.0,
                         sense=pyo.maximize)

# Constraints
model.Constraint3 = pyo.Constraint(
    expr=3.0 * model.x['drs'] + 2.0 * model.x['wdw'] <= 18)

# Solve
solver = pyo.SolverFactory('ipopt')
solver.solve(model)

# display(model)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("Profit = ",
      numbers.format_currency(1000 * model.profit(), 'USD', locale='en_US'))
print("Batches of Doors = {:1.2f}".format(model.x['drs']()))
print("Batches of Windows = {:1.2f}".format(model.x['wdw']()))

# Local Search - Continuous Variables

Local search algorithms for continuous variables are generally based on approximating the objective function near the current search point, then using that approximation to compute an improved search point.  For instance if we can calculate the gradient (calculus) or approximate it, then a move along the gradient direction will increase the value of the function.  

We'll primarily use the `scipy.optimize` function `minimize` for local search on continuous functions.  You can read more about it below.  

## Minimize from scipy.optimize

In [ ]:
# execute this cell to see scipy.optimize.minimize documentation
from IPython.display import IFrame
IFrame(
    "https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize",
    width=900,
    height=600)

While there are many options here, the defaults will serve well for our purposes.  The `method` specifies a variety of different numerical algorithms for local search.  We'll use `BFGS` for unbounded problems and `L-BFGS-B` for problems with bounds.  You shouldn't have to specify that choice as those are the defaults.  The BFGS methods are robust algorithms and are known as quasi-Newton methods.  What this means is that they approximate the shape of the objective function near the current point by approximating the derivatives (slopes and curvature) of the function and that shape information is used to produce an improved search point. 

One of the things to pay attention to here is how we have to write our objective functions so that they can be passed to the `minimize` function.

### A univariate function

Lets investigate the fourth degree polynomial $$p(x) = x^4 + 2 x^3 + 3 x^2 + 2 x + 1.$$  Let's graph it to get an idea of the behavior. 

In [ ]:
# plot p(x) on [-10,10]
x = np.linspace(-10,10,201)
p = lambda x:x**4 + 2*x**3 + 3*x**2 + 2*x + 1
fig = plt.figure(figsize=(4,3.5))
plt.plot(x,p(x));
plt.xlabel('x');
plt.ylabel('y');

The overall "U" shape is not surprising since for polynomials the behavior for large values of $x$ is determined by the highest degree term which is, in this case, $x^4$.  It appears that there is a minimum or minima close to the origin.  Let's zoom in a bit to see what we can:

In [ ]:
# plot p(x) on [-3,3]
x = np.linspace(-3,3,201)
p = lambda x:x**4 + 2*x**3 + 3*x**2 + 2*x + 1
fig = plt.figure(figsize=(4,3.5))
plt.plot(x,p(x));
plt.xlabel('x');
plt.ylabel('y');

There appears to be only one minimum somewhere around $x=0$ or $x=-1$.  Since the function appears to be convex, the starting point doesn't matter.  Let's search for the minimum beginning at $x_0 = -2$:

In [ ]:
# execute for local search
result = minimize(p,-2)
result

In [ ]:
print('The minimum value is {:0.4f} and occurs at x = {:0.4f}'.format(result.fun,result.x[0]))

###  <font color = "blue">Self Assessment:  Minimize to Maximize</font>

An apartment complex has 250 apartments to rent and that their profit in thousands of dollars is given by the function 
$$P(x) = -0.008 x^2 + 3.1 x - 80.$$
Find the maximum profit and how many apartments to rent to achieve the maximum profit.  Use minimize from scipy.optimize to find the maximum.  Review the video above to see how to "flip" the problem to find a maximum.

In [ ]:
# graph of profit function
x = np.linspace(0,250,201) # 201 points between 0 and 250
P = lambda x:-1*(-0.008*x**2 + 3.1*x - 80) # lambda is for writing one line functions
fig = plt.figure(figsize=(4,3.5));
plt.plot(x,P(x));
plt.xlabel('apartments');
plt.ylabel('profit (\$ thousands)');

In [ ]:
# execute for local search
result = minimize(P,150)
print(f"{-1*result.fun},{result.x[0]}")

Your answer shouldn't be an integer.  We could use discrete optimization and only optimize integer numbers of apartments, but it will usually be more computationally intensive.  Instead we're using a continuous variable to get an approximation to the discrete problem, this is called **relaxation** (we've relaxed the integer variable condition).  So what whole number of apartments should you rent?  Why? 

### <font color = "blue">Self Assessment:  Finding Multiple Extrema</font>

The function $f(x) = x^5 - x^4 - 18 x^3 + 16 x^2 + 32 x - 2$ for $-4 \leq x \leq 3.6$ appears in the video "Gradient Descent and Local Minima" above.  Plot the function on the given interval.  Use the graph to guess where the local maxima and minima are.  Now use minimize from scipy.optimize to find the $x$ and $y$ coordinates of all the extrema.  If you supply a bounds argument to `minimize` you may get unexpected results, instead use the graph to make reasonable guesses to the location of the optima.

## Example: Simple Logistic Regression

### Video Walkthrough of this example

In [ ]:
# execute this cell for video
from IPython.display import IFrame
IFrame(
    "https://media.uwex.edu/content/ds/ds775_r19/ds775_lesson4-logistic-regression/index.html",
    width=640,
    height=360)

### Setup for Logistic Regression

Most machine learning algorithms are driven by optimization.  Usually we want to minimize a loss function which measures the difference between the model predictions and the observed data.  Neural network training uses a version of the gradient descent algorithm to optimize the weights in the network.  Here we'll show how to fit a logistic regression model by maximizing a function.

In simple logistic regression we try to predict the value of the label $y$, which can be 0 or 1, for each value of a continuous predictor variable $x$.  In particular, the conditional probability that $y=1$ given the current value of $x$ is modeled by a sigmoid function ("s" curve) $$p(x) = \frac{1}{1 + e^{-(b_0 + b_1 x)}}.$$

For example, the more hours a student studies to prepare for an exam, the higher the probability that they will pass the test.  Shown below is some data.  For each student we have the number of hours they studied and whether or not they passed the exam (1 for passed, 0 for failed).  This example data comes from the <a href="https://en.wikipedia.org/wiki/Logistic_regression">Wikipedia article on Logistic Regresssion</a>.

### An example

In [ ]:
# data from 
x_hours = np.array([
    0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 1.75, 2.0, 2.25, 2.50, 2.75, 3.00, 3.25,
    3.5, 4.0, 4.25, 4.5, 4.75, 5.0, 5.5
])
y_passed = np.array([0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,1,1,1,1,1])

Below is a graph of the data along with the graph of the fitted sigmoid function that models the probality of $y=1$ at each $x$.  Don't worry, we'll see where the fitted curve comes from in a bit.

In [ ]:
# graph of data and sigmoid
b0 = -4.07771657
b1 = 1.5046468
hours_studied = np.linspace(0,6,101)
def sigmoid(x,intercept,slope):
    return( 1.0 / (1.0 + np.exp( -(intercept + slope * x) ) ) )
prob_passed = sigmoid(hours_studied, b0, b1)

fig = plt.figure();
fig.set_size_inches(6,3.5);
ax = fig.add_subplot(111);
ax.scatter(x_hours, y_passed);
ax.plot(hours_studied, prob_passed);
ax.set_xlabel('hours studied');
ax.set_ylabel('passed');

We can use the model to make predictions:

In [ ]:
print(
    'A student who studies 4 hours has approximately {:3.1f}% chance of passing.'
    .format(100 * sigmoid(4, b0, b1)))

*Note:  It isn't important to understand the details of logistic regression for this class, but the text in this cell gives a bit of background.*

Where do those values for the slope and intercept come from?  To obtain those we find the values of $b_0$ and $b_1$ that maximize the likelihood function:
$$ L(b_0,b_1) = \prod_{i=1}^{n} p(x_i)^{y_i} (1-p(x_i))^{(1-y_i)}$$
where $(x_i,y_i)$ are the data pairs for each student and $p(x) = \displaystyle \frac{1}{1 + e^{-(b_0 + b_1 x)}}$ is the sigmoid function.  By maximizing the likelihood function we are maximizing the probability that this model produced the observed data.  Note that the $\prod$ symbol means to take the product of the values, like $\sum$ means to take the sum.

In practice, maximizing a product can lead to numerical difficulties, so we instead maximize the log-likelihood function found by taking the logarithm of $L(b_0,b_1)$ to get:
$$LL(b_0, b_1) = \sum_{i = 1}^{n} \left[ y_i \log( p(x_i) ) + (1-y_i) \log(1-p(x_i)) \right].$$

Now we need to find $b_0$ and $b_1$ to maximize this.  The log-likelihood function turns out to be concave so that ascending from any starting point will lead to the global maximum.  

Because we will use the `minimize` function from `scipy.optimize` to find the maximum log-likelihood we'll minimize the negative log-likelihood:

### Find the model with `minimize`

In [ ]:
def neg_log_loss( coef, *args):
    b0 = coef[0]
    b1 = coef[1]
    x = args[0]
    y = args[1]
    p = 1.0/(1.0 + np.exp(-(b0 + b1*x)))
    ll = sum( y*np.log(p)+(1-y)*np.log(1-p) )
    return(-ll) # here's the minus sign!

Take a minute to see how this function is structured and perhaps glance at the documentation again.  `coef` is a one-dimensional array with shape (n,) that contains all $n$ optimization variables.  In this case there are two which we assign to $b_0$ and $b_1$.  `*args` is a pointer to tuple `args` that contains any additional parameters that should be passed to the function.  In the case `minimize` will be passing the tuple $(x,y)$ that contains the training data.  We'll pass `args = (x_hours, y_passed)`.

In [ ]:
result = minimize(neg_log_loss,[0,0],args=(x_hours,y_passed))
result

In [ ]:
b0 = result.x[0]
b1 = result.x[1]
print('The maximum likelihood estimate for p(x) has intercept b0 = {:2.3f} and slope b1 = {:2.3f}'.format(b0,b1))

Note:  the approach outlined here for logisitic regression is very similar to the actual algorithms used by most software for computing logistic regression models. Many machine learning predictive models are trained by optimization.  To verify our results we check our results against those from Sci-kit Learn.  By default sklearn uses an L2 regularization term to avoid overfitting (more about this in DS740).  The amount of regularization is proportional to $1/C$ so we just use a huge $C$ to mimic no regularization.

In [ ]:
from sklearn.linear_model import LogisticRegression    
model = LogisticRegression(C=1.0e10,fit_intercept = True)
model.fit(x_hours.reshape(-1,1), y_passed)
b0 = model.intercept_[0]
b1 = model.coef_[0][0]
print('The maximum likelihood estimate for p(x) has intercept b0 = {:2.3f} and slope b1 = {:2.3f}'.format(b0,b1))

## Example:  The Rastrigin Function

The Rastrigin function is a common test case for optimization algorithms because it has many local minima.  The definition of the function is 
$$f(\mathbf{x})=10 n+\sum_{i=1}^{n}\left[x_{i}^{2}-A \cos \left(2 \pi x_{i}\right)\right]$$
Where $n$ is the dimensionality of input vector $\mathbf{x}$.  For instance if $n=2$ then $\mathbf{x} = (x_1, x_2)$.  The domain is restricted so that each $x_i \in [-5.12, 5.12].$ .   Here is a graph of the the Rastrigin function with dimension $n=1.$

In [ ]:
# open to reveal graph code
def rastrigin_1D(x):
    return (x**2 + 10 - 10 * np.cos(2 * np.pi * x))
    
x = np.linspace(-5.12,5.12,201)
y = rastrigin_1D(x)

fig = plt.figure(figsize=(3,3))
plt.plot(x,y)
plt.xlabel('x');
plt.ylabel('y');

In [ ]:
# 3D Graph of Rastrigin with dimension n = 2
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np

x = np.linspace(-5.12, 5.12, 401)     
y = np.linspace(-5.12, 5.12, 401)     
X, Y = np.meshgrid(x, y) 
Z = (X**2 - 10 * np.cos(2 * np.pi * X)) + \
  (Y**2 - 10 * np.cos(2 * np.pi * Y)) + 20

data = [
    go.Surface( x = X, y = Y, z = Z, colorscale = 'Jet',
        contours=go.surface.Contours(
            z=go.surface.contours.Z(
              show=True,
              usecolormap=True,
              highlightcolor="#42f462",
              project=dict(z=True)
            )
        )
    )
]

layout = go.Layout(title='Rastrigin',width=600,height=600)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

The Rastrigin function isn't important as a real-life example, but it does serve as a good test problem with oodles of local minima and we know that global minimum occurs at the origin.  This is similar to what can happen in training in neural networks and other complex models except that we don't know where the global optimum is.

A simple approach for trying to find the global minimum of a multi-modal function is called a **restart** or **multistart strategy** in which local searches are started at randomly generated initial points and the most optimal result of all the local searches is recorded.

Here is pseudo-code for a multistart code:
```
for num_searches:
 choose random initial state
 do local search
 if new optimum
     remember it
endfor
```


In [ ]:
def rastrigin(x):
    # pass a single vector of length n (=dim) to evaluate Rastrigin
    return sum(x**2 + 10 - 10 * np.cos(2 * np.pi * x))

dim = 10
num_local_searches = 1000
best_value = 1.e10

for i in range(num_local_searches):
    x_initial = np.random.uniform(-5.12, 5.12, dim)
    result = minimize(rastrigin, x_initial)
    if result.fun < best_value:
        best_value = result.fun
        best_x = result.x

print(
    'The smallest value find is {:4.3f} at x = {:1.3f} and y = {:1.3f}'.format(
        best_value, best_x[0], best_x[1]))

### <font color="blue">Self-Assessment: Rastrigin with dim = 3, 4</font>

How many iterations does it take to reliably find the global minimum with dim = 3?  With dim = 4?  Use the multi-start strategy.

### <font color = "blue">Self-Assessment:  Rastrigin with dim = 10 </font>

Do 1000 local search with Rastrigin with dim = 10.  What is the smallest value you find?  How long do you think it would take to find the minimum from randomly chosen initial points like this?  

## The curse of dimensionality

If we are maximizing a function of one variable, $f(x)$, we might choose to use 10 starting points.  For a function of two variables, $g(x,y)$ to get the same search power we would choose 10 points in the $x$ direction and 10 points in the $y$ direction to make a grid of $10^2 = 100$ starting points in the $xy$-plane.  For three variables we need $10^3 = 1000$ points in $xyz$-space.  For a function of $n$ variables we would need $10^n$ starting points.

*The volume of the search space grows exponentially with the number of variables or dimensionality of the problem.*

This is called the curse of dimensionality.  For high dimensional functions like those that occur in training neural networks and other applications with many local minima it can be very difficult to find the global minima because the volume of the search space grows exponentially with the number of variables.

For the Rastrigin function to find the global minimum you need an initial starting point in the interval (-0.5,0.5) in each dimension.  The search interval is [-5.12,5.12] in each dimension.  Thus the probability that a single uniformly sampled point in [-5.12,5.12] is $\frac{1}{10.28} \approx 0.0973$ (the ratio of the lengths of the two intervals).  The probability of finding the global minimum using local search from a uniformly sampled point in $n$ dimensions is $$\left( \frac{1}{10.28} \right)^n$$.  That means we'd have to, on average, start $10.28^n$ local searches from uniformly sampled points to find the global minimum once.

### <font color="blue">Self-Assessment:  How many searches?</font>

For the Rastrigin function write a while loop that runs until the global minimum value is found ($|\mbox{best_val}|<0.01$) and track the number of iterations. Use a for loop to repeat this three times until your code is debugged.  After your code is working, repeat the process 100 times and report the average number of searches until the global minimum is found when $n=1,2,3$.  Are these numbers in approximate agreement with with the estimated numbers $10.28^n$ (they very likely won't be all that close, but how is the overall trend)?

### <font color = "blue">Self Assessment: How many searches when dim = 10?</font>

Approximately now many local searches are required to find the global minimum one time when dim = 10?  Is it surprising that you (very likely) didn't find it with 1000 local searches?  Explain

# Local Search - Discrete Variables

Optimization with discrete variables tends to be more complicated than with continuous variables.  In the continuous case we can take advantage of calculus or numerical methods to compute gradient search directions that allow us to move to nearby points that are closer to optimal.  However with discrete random variables there is no generic way to compute better nearby points.  Often the best we can do is find nearby points, which is usually problem specific, and try them to see if they produce closer to optimal results.

We'll look carefully at the traveling salesman problem (TSP).  In addition to the information about the TSP in the textbook, there is copious information available on the internet.

## The Basics of Local Search

Here is pseudo-code for a simple local search.  Many variations are possible, but they often look like this:
```
 set starting state 
 while local_condition 
     select a move 
     if acceptable 
         do the move 
         if new optimum 
             remember it 
 endwhile 
 ```

Notes:

* Often the starting state is one selected at random.  State refers to the "state" or values of the variables.

* The local condition is a stopping condition.  It could be something like stopping after a fixed number of iterations or stopping after making no or insignificant process for a while.

* Selecting a move is where things get problem specific.  Often the move involves a random change to the variables.

* If acceptable means that we are checking to see that the state is feasible, that is, does it satisfy the constraints?

## Local Search for TSP

Let's see what this looks like for the TSP.  We'll use the subtour reversal algorithm, described in the textbook, to generate moves.  Here's what the local search looks like for the TSP:

```
 choose a random tour  
 while shorter tours have been found in last max_tries
     propose new tour with one random segment reversed
     if acceptable (it will always be a valid tour) 
         compute new tour distance 
         if new shortest tour 
             remember it 
         else
             reject new tour
 endwhile 
 ```

## Video for TSP Local Search Code

In [ ]:
# execute this cell for video
from IPython.display import IFrame
IFrame(
    "https://media.uwex.edu/content/ds/ds775_r19/ds775_lesson4-logistic-traveling-salesman-problem/index.html",
    width=640,
    height=360)

## TSP Local Search Code

We'll be minimizing the total length of a tour that visits all 48 state capitols in the continental United States and ends back in the same city in which it begins.  The latitudes and longitudes of the cities are projected onto a rectangular coordinate system with $x$ and $y$ coordinates representing positions in meters which we convert to kilometers.  We use the `Matplotlib Basemap` package to do the projections and to draw routes on a map of the United States.  If you're trying to install Basemap at home make sure you're doing in a virtual environment that is not your base environment (as mentioned before, create a ds775 environment with conda).  Basemap doesn't install correctly, at least on macs, in the base environment.

First we load the data and do the projections.  The $x$ and $y$ coordinates of the cities are stored in the 48 x 2 array `xy` with one pair per row.  We also define a function to visualize the tours.

In [ ]:
# load the data and define move and objective functions
map = Basemap(llcrnrlon=-119,
              llcrnrlat=22,
              urcrnrlon=-64,
              urcrnrlat=49,
              projection='lcc',
              lat_1=32,
              lat_2=45,
              lon_0=-95)

# read 48 capitals lat and lon
with open('./data/capitals48.json', 'r') as json_file:
    capitals = json.load(json_file)

# loop through the capitals to store x,y coordinates (meters)
num_cities = len(capitals)
xy = np.zeros((num_cities, 2))
for i in range(num_cities):
    lat = capitals[i]["lat"]
    lon = capitals[i]["long"]
    xy[i, :] = map(lon, lat)

def plot_tour(best_tour, xy, best_dist):
    fig = plt.figure()
    fig.set_size_inches(6, 4)

    # load the shape file with "states"
    map.readshapefile('./data/st99_d00', name='states', drawbounds=True)

    loop_tour = np.append(best_tour, best_tour[0])
    map.plot(xy[:, 0], xy[:, 1], c='r', marker='o', markersize=4, linestyle='')
    lines, = map.plot(xy[loop_tour, 0],
                      xy[loop_tour, 1],
                      c='b',
                      linewidth=1,
                      linestyle='-')
    plt.title('Best Distance {:d} km'.format(int(best_dist)))

Here we define both the local "move" function which reverses a tour segment to generate a new tour and the objective function which computes the length of the tour in kilometers:

In [ ]:
# define move and objective functions

def sub_tour_reversal(tour):
    n = len(tour)
    i, j = np.sort(np.random.choice(n, 2, replace=False))
    return (np.concatenate((tour[0:i], tour[j:-n + i - 1:-1], tour[j + 1:n])))

def tour_dist(tour, xy):
    # returns tour distance in kilometers
    xy_tmp = xy[tour, :]
    return (np.sum(
        np.sqrt(
            np.sum(np.diff(xy_tmp, axis=0, append=[xy_tmp[0, :]])**2, axis=1)))
            / 1000)

In [ ]:
# local search for TSP solution with a random segment reversal at each iteration

def random_reversals(xy, max_no_improve):
    num_cities = xy.shape[0]
    # starts from a random tour
    current_tour = np.random.permutation(np.arange(num_cities))
    current_dist = tour_dist(current_tour, xy)
    best_tour = current_tour
    best_dist = current_dist

    # stop search if no better tour is found within max_no_improve iterations, can increase to eliminate crossovers
    num_moves_no_improve = 0
    iterations = 0
    while (num_moves_no_improve < max_no_improve):
        num_moves_no_improve += 1
        iterations += 1  # just for tracking
        new_tour = sub_tour_reversal(current_tour)
        new_dist = tour_dist(new_tour, xy)
        if new_dist < current_dist:
            num_moves_no_improve = 0
            current_tour = new_tour
            current_dist = new_dist
            if current_dist < best_dist:  # not really needed since current_tour will be best
                best_tour = current_tour  # but we'll use this in the next lesson
                best_dist = current_dist
    return best_tour, best_dist, iterations


best_tour, best_dist, iterations = random_reversals(xy, 1000)
plot_tour(best_tour, xy, best_dist)
print('The minimum distance found is {:d} after {:d} iterations'.format(
    int(best_dist), iterations))

Though it's not important, if you like you can execute the two cells below to see an animated version of this search:

In [ ]:
# execute first to initialize the plot
fig = plt.figure()
fig.set_size_inches(6,4)
fig.canvas.draw()
fig.show()

In [ ]:
# execute to watch the progress of random reversals local search

map.readshapefile('./data/st99_d00', name='states', drawbounds=True)

# initialize with a random tour
n = 48
best_tour = np.random.permutation(np.arange(n))
best_dist = tour_dist(best_tour, xy)/1000

# plot initial tour
loop_tour = np.append(best_tour, best_tour[0])
map.plot(xy[:, 0], xy[:, 1], c='r', marker='o', markersize=4, linestyle='')
lines, = map.plot(xy[loop_tour, 0],
                  xy[loop_tour, 1],
                  c='b',
                  linewidth=1,
                  linestyle='-')
fig.canvas.draw()
plt.show()

count = 1
iteration = np.array([count])
distances = np.array([best_dist])
dst_label = plt.text(250000, 650000, '{:d} km'.format(int(best_dist)))


# initialize with a random tour
current_tour = np.random.permutation(np.arange(num_cities))
current_dist = tour_dist(current_tour, xy)
best_tour = current_tour
best_dist = current_dist

max_moves_no_improve = 5000
num_moves_no_improve = 0
while( num_moves_no_improve < max_moves_no_improve):
    num_moves_no_improve += 1
    new_tour = sub_tour_reversal(current_tour)
    new_dist = tour_dist(new_tour, xy)/1000
    if new_dist < current_dist:
        current_tour = new_tour
        current_dist = new_dist
        num_moves_no_improve = 0
        if current_dist < best_dist: # not really needed since current_tour will be best
            best_tour = current_tour # but we'll use this in the next lesson
            best_dist = current_dist
        loop_tour = np.append(best_tour, best_tour[0])
        lines.set_data(xy[loop_tour, 0], xy[loop_tour, 1])
        dst_label.set_text('{:d} km'.format(int(best_dist)))   
        fig.canvas.draw()
        plt.show()  


## Another local search algorithm for TSP

We'll have a look at an algorithm called "2-opt" that was proposed by Croes in 1958.  We won't focus on it too much since the idea doesn't really extend to other problems.  The main idea  is to reverse segments that cross over themselves to remove the cross over.  We loop repeatedly over all the possible reversals until there are no more cross overs.  Here is some Python to do 2-opt:

In [ ]:
# 2-opt local search for TSP

def two_opt(xy):
    num_cities = xy.shape[0]
    current_tour = np.random.permutation(np.arange(num_cities))
    current_dist = tour_dist(current_tour, xy)
    best_tour = current_tour
    best_dist = current_dist

    improvement = True
    iterations = 0
    while improvement:
        improvement = False
        for i in range(num_cities - 1):
            for j in range(i + 1, num_cities):
                iterations += 1
                new_tour = sub_tour_reversal(best_tour, i, j)
                new_dist = tour_dist(new_tour, xy)
                if new_dist < best_dist:
                    best_tour = new_tour
                    best_dist = new_dist
                    improvement = True
    return best_tour, best_dist, iterations

best_tour, best_dist, iterations = two_opt(xy)
                
plot_tour(best_tour,xy,best_dist)
print('The minimum distance found is {:d} after {:d} iterations'.format(int(best_dist),iterations))

2-opt is generally uses fewer iterations to find a reasonable tour (local minimum) than does our local search with random segment reversals.  In the next lesson we'll try using 2-opt to find starting points for a global search algorithm.

### Local Search with Graphing for TSP

In [ ]:
# execute this cell before those below to initialize the figure display, important for high-res displays
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from mpl_toolkits.basemap import Basemap
import json
import numpy as np
import time

fig = plt.figure()
fig.set_size_inches(6,4)
fig.canvas.draw()
fig.show()

In [ ]:
# create the map and run the 2opt search
map = Basemap(llcrnrlon=-119,
              llcrnrlat=22,
              urcrnrlon=-64,
              urcrnrlat=49,
              projection='lcc',
              lat_1=32,
              lat_2=45,
              lon_0=-95)

# load the shape file with "states"
map.readshapefile('./data/st99_d00', name='states', drawbounds=True)

# read 48 capitals lat and lon
with open('./data/capitals48.json', 'r') as json_file:
    capitals = json.load(json_file)
num_cities = 48

# loop through the capitals to store x,y coordinates (meters)
xy = np.zeros((num_cities, 2))
for i in range(num_cities):
    lat = capitals[i]["lat"]
    lon = capitals[i]["long"]
    xy[i, :] = map(lon, lat)


def sub_tour_reversal(tour, i, j):
    # reverse the segment from city i to city j
    n = len(tour)
    return (np.concatenate((tour[0:i], tour[j:-n + i - 1:-1], tour[j + 1:n])))


def tour_dist(tour, xy):
    # euclidean distance returned
    xy_tmp = xy[tour, :]
    return (np.sum(
        np.sqrt(
            np.sum(np.diff(xy_tmp, axis=0, append=[xy_tmp[0, :]])**2,
                   axis=1))))


# initialize with a random tour
n = 48
best_tour = np.random.permutation(np.arange(n))
best_dist = tour_dist(best_tour, xy)/1000
improvement = True

# plot initial tour
loop_tour = np.append(best_tour, best_tour[0])
map.plot(xy[:, 0], xy[:, 1], c='r', marker='o', markersize=4, linestyle='')
lines, = map.plot(xy[loop_tour, 0],
                  xy[loop_tour, 1],
                  c='b',
                  linewidth=1,
                  linestyle='-')
fig.canvas.draw()
plt.show()

count = 1
iteration = np.array([count])
distances = np.array([best_dist])
dst_label = plt.text(250000, 650000, '{:d} km'.format(int(best_dist)))


# initialize with a random tour
current_tour = np.random.permutation(np.arange(num_cities))
current_dist = tour_dist(current_tour, xy)
best_tour = current_tour
best_dist = current_dist

max_moves_no_improve = 5000
num_moves_no_improve = 0
while( num_moves_no_improve < max_moves_no_improve):
    num_moves_no_improve += 1
    i,j = np.sort(np.random.choice(n,2,replace=False))
    new_tour = sub_tour_reversal(current_tour, i, j)
    new_dist = tour_dist(new_tour, xy)/1000
    if new_dist < current_dist:
        current_tour = new_tour
        current_dist = new_dist
        num_moves_no_improve = 0
        if current_dist < best_dist: # not really needed since current_tour will be best
            best_tour = current_tour # but we'll use this in the next lesson
            best_dist = current_dist
        loop_tour = np.append(best_tour, best_tour[0])
        lines.set_data(xy[loop_tour, 0], xy[loop_tour, 1])
        dst_label.set_text('{:d} km'.format(int(best_dist)))   
        fig.canvas.draw()
        plt.show()  


## Gerrymandering Example

This example is based on textbook problem 13.10-6 which is reproduced here:

Because of population growth, the state of Washington has been given an additional seat in the House of Representatives, making a total of 10. The state legislature, which is currently controlled by the Republicans, needs to develop a plan for redistricting the state. There are 18 major cities in the state of Washington that need to be assigned to one of the 10 congressional districts. The table below gives the numbers of registered Democrats and registered Republicans in each city. Each district must contain between 150,000 and 350,000 of these registered voters. Assign each city to one of the 10 congressional districts in order to maximize the number of districts that have more registered Republicans than registered Democrats.

<img src="images/gerrymandering.png" width="300">

We'll provide the data and objective function below.

In [ ]:
# imports
import pandas as pd 
import numpy as np

In the cell below we load the data and create an assignment of the 18 cities to the 10 districts.

In [ ]:
# load the data + random assignment
num_districts = 10
min_voters_in_district = 150
max_voters_in_district = 350

dems = [152,81,75,34,62,38,48,74,98,66,83,86,72,28,112,45,93,72]
reps = [62,59,83,52,87,87,69,49,62,72,75,82,83,53,98,82,68,98]
cities = pd.DataFrame( data = {'dems':dems, 'reps':reps})

# assign = np.random.randint(low=0,high=num_districts,size = 18)
assign = np.array([4, 3, 1, 9, 4, 0, 2, 8, 0, 9, 8, 0, 0, 0, 8, 7, 3, 6])
assign

The `summarize_districts` function below isn't used directly in the initialization, but it helps us by printing out the number of voters (in thousands) assigned to each district and shows us which districts are won by republicans.

In [ ]:
def summarize_districts(assign, cities):
    reps = np.zeros(num_districts, dtype=np.int32)
    dems = np.zeros(num_districts, dtype=np.int32)
    df = cities.groupby(assign).sum()
    reps[df.index] = df['reps']
    dems[df.index] = df['dems']
    total = reps + dems
    delta = np.minimum(np.maximum(total, min_voters_in_district),
                       max_voters_in_district) - total
    rep_win = reps > dems
    dict = {
        'reps': reps,
        'dems': dems,
        'total': total,
        'rep_win': rep_win
    }
    return (pd.DataFrame(data=dict))

summarize_districts(assign, cities)

For the objective function, `fitness_districts` we count the number of districts won by republicans.  Instead of checking each potential solution to see if it's feasible (between 150 and 350 voters in each district) we subtract the total number of thousands by which each district is out of bounds.  This approach to optimization is called a **penalty function method**.  It doesn't prevent infeasible solutions, but it strongly penalizes them so that when the function is optimized the solution generally will be feasible.  Note that in the table above we have several districts that too many or two few voters, when we compute the fitness of that assignment to districts it is negative because of the penalty term.

In [ ]:
def fitness_districts(assign, cities):
    df = cities.groupby(assign).sum()
    fitness = sum( df['reps'] > df['dems'] )
    total_voters = np.zeros(num_districts,dtype=np.int32)
    total_voters[df.index] = df.sum(axis=1)
    fitness -= np.abs(np.minimum(np.maximum(total_voters,150),350)-total_voters).sum()
    return (fitness)

fitness_districts(assign,cities)

### <font color = "blue"> Self Assessment: Gerrymandering Local Search</font>

Start with a random state and move to a nearby state by changing one of the city assignments randomly.  You don't have to check for feasibility of each new state, just accept the move if you get a larger value of the fitness function.  Repeat until no progress is made for 1000 moves.  Your local search should be inside of a function with inputs the cities and the initial assignment.  The output should be the optimized assignment and the value of the fitness function.  

Now write a loop that does 100 local searches.  Make sure the final, best solution is feasible.  What is the maximum number of districts that Republicans win?